# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


### ① 印出最新文章的「作者」「標題」「時間」

In [2]:
#個人備註
#先用成btfs可剖析的變數
#==============================================
from bs4 import BeautifulSoup as btfs
import requests

url ='https://www.ptt.cc/bbs/NBA/index.html'
r = requests.get(url)
soup = btfs(r.text,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   看板 NBA 文章列表 - 批踢踢實業坊
  </title>
  <link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>
  <link href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="//images.ptt.cc/bbs/v2.27/bbs-custom.css" rel="stylesheet" type="text/css"/>
  <link href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <div id="topbar-container">
   <div class="bbs-content" id="topbar">
    <a href="/bbs/" id="logo">
     批踢踢實業坊
    </a>
    <span>
     ›
    </span>
    <a class="board" href="/bbs/NBA/index.html">
     <span class="board-label">
      看板
     </span>
     NBA
    </a>
    <a class="right small" href="/about

In [101]:
#個人備註
#用pandas顯示
#==============================================
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

empty=[]
for i in soup.find_all('div',class_='r-ent'):
    
    #下面這一行很關鍵，因為我們如果不進去翻內頁，他的日期就只是X/X，沒有更詳細的時間
    #所以設定一個新的btfs可剖析的變數，他在for迴圈裡得以成功是因為他的內文連結在原始碼中
    #剛好可以一個一個翻出來，再append到空list裡
    #但感覺運行起來有點慢，因為要爬取大量的網頁
    
    new_r=btfs(requests.get('https://www.ptt.cc/'+i.find('a')['href']).text,'lxml')
    
    #因為發現他會把\t\n這些空行換行指令弄進來，所以要弄個replace('unwanted','wanted')
    title=i.find('div',class_='title').text.replace('[', '').replace(']', '')
    
    
    #下面這個就是從每個內文裡面去抓他的作者資訊，比原本的詳盡，不知道他是否有活動之類的，
    #他有些文章格式跟其他不一樣，避免整個迴圈出現錯誤，用try,except
    #其實這邊用new_r.find就好了，因為作者就是第1項，但考慮到公整，跟下面用一樣的方法
    detailed_author=new_r.find_all('span',class_='article-meta-value')
    try:                          
        author=detailed_author[0].text.replace('[', '').replace(']', '')
    except:
        author='NaN'
    
    #下面這個就是從每個內文裡面去抓他的日期，比原本的詳盡，不知道他是否有活動之類的，
    #他有些文章格式跟其他不一樣，避免整個迴圈出現錯誤，用try,except
    detailed_date=new_r.find_all('span',class_='article-meta-value')
    try:                       
        date=detailed_date[3].text.replace('[', '').replace(']', '')
    except:
        date='NaN'
    
    #最後就是新增到空list
    empty.append([date,author,title])

    
columns=['日期','作者','標題']#改名
data=pd.DataFrame(empty,columns=columns)#改名
data

,日期,作者,標題
0,Mon Mar 9 11:34:29 2020,sthho (sthho),\n討論 NBA停賽的可能性?!\n
1,NaN,NaN,\nBOX Raptors 118:113 Kings 數據\n
2,Mon Mar 9 11:44:26 2020,Vedan (味丹),\n公告 第一次被退文，可在三天後刪除退文\n
3,Mon Mar 9 11:44:29 2020,sodistrues (true),\n新聞全陣快艇吞第一敗 湖人本季洛城內戰首開\n
4,Mon Mar 9 11:46:59 2020,NSYSUheng (Pain),\n情報 雷霆本季16次前三節落後逆轉勝 聯盟最多\n
5,Mon Mar 9 11:59:13 2020,MayBeSoft (小小瑜兒),\n討論 快艇是不是被騙了買到假貨啊\n
6,Mon Mar 9 12:55:19 2020,UCAthena (Let's go __),\n花邊 球迷嗆Walton：你這白痴，你有三分冠軍\n
7,Mon Mar 9 14:05:10 2020,IAMGRICE (重雙我),\nRe: 花邊 歐尼爾:誰能擊敗這五個人！\n
8,Mon Mar 9 14:26:53 2020,ericgary (艾瑞克蓋瑞),\nRe: BOX Lakers 112:103 Clippers 數據\n
9,Mon Mar 9 15:11:57 2020,manuginobii (做愛校 果真累人),\n情報 28次失誤?籃網臨時教練:前7個是球有點滑\n


In [103]:
#理應最底的是最新文章，但18-22為一些公告，所以經過個人篩選後，是以下這一篇
data.iloc[17]

日期          Mon Mar  9 19:13:14 2020
作者               DarkApplePie (黑林檎派)
標題    \nRe: 討論 LBJ現今擁有的成就已經能超過賈霸了嗎\n
Name: 17, dtype: object

### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [102]:
#個人備註
#用pandas顯示
#==============================================
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

empty=[]
for i in soup.find_all('div',class_='r-ent'):
    
    #下面這一行很關鍵，因為我們如果不進去翻內頁，他的日期就只是X/X，沒有更詳細的時間
    #所以設定一個新的btfs可剖析的變數，他在for迴圈裡得以成功是因為他的內文連結在原始碼中
    #剛好可以一個一個翻出來，再append到空list裡
    #但感覺運行起來有點慢，因為要爬取大量的網頁
    
    new_r=btfs(requests.get('https://www.ptt.cc/'+i.find('a')['href']).text,'lxml')
    
    #因為發現他會把\t\n這些空行換行指令弄進來，所以要弄個replace('unwanted','wanted')
    title=i.find('div',class_='title').text.replace('[', '').replace(']', '')
    
    
    #下面這個就是從每個內文裡面去抓他的作者資訊，比原本的詳盡，不知道他是否有活動之類的，
    #他有些文章格式跟其他不一樣，避免整個迴圈出現錯誤，用try,except
    #其實這邊用new_r.find就好了，因為作者就是第1項，但考慮到公整，跟下面用一樣的方法
    detailed_author=new_r.find_all('span',class_='article-meta-value')
    try:                          
        author=detailed_author[0].text.replace('[', '').replace(']', '')
    except:
        author='NaN'
    
    #下面這個就是從每個內文裡面去抓他的日期，比原本的詳盡，不知道他是否有活動之類的，
    #他有些文章格式跟其他不一樣，避免整個迴圈出現錯誤，用try,except
    detailed_date=new_r.find_all('span',class_='article-meta-value')
    try:                          
        date=detailed_date[3].text.replace('[', '').replace(']', '')
    except:
        date='NaN'
    
    #最後就是新增到空list
    empty.append([date,author,title])

    
columns=['日期','作者','標題']#改名
data=pd.DataFrame(empty,columns=columns)#改名
data

,日期,作者,標題
0,Mon Mar 9 11:34:29 2020,sthho (sthho),\n討論 NBA停賽的可能性?!\n
1,NaN,NaN,\nBOX Raptors 118:113 Kings 數據\n
2,Mon Mar 9 11:44:26 2020,Vedan (味丹),\n公告 第一次被退文，可在三天後刪除退文\n
3,Mon Mar 9 11:44:29 2020,sodistrues (true),\n新聞全陣快艇吞第一敗 湖人本季洛城內戰首開\n
4,Mon Mar 9 11:46:59 2020,NSYSUheng (Pain),\n情報 雷霆本季16次前三節落後逆轉勝 聯盟最多\n
5,Mon Mar 9 11:59:13 2020,MayBeSoft (小小瑜兒),\n討論 快艇是不是被騙了買到假貨啊\n
6,Mon Mar 9 12:55:19 2020,UCAthena (Let's go __),\n花邊 球迷嗆Walton：你這白痴，你有三分冠軍\n
7,Mon Mar 9 14:05:10 2020,IAMGRICE (重雙我),\nRe: 花邊 歐尼爾:誰能擊敗這五個人！\n
8,Mon Mar 9 14:26:53 2020,ericgary (艾瑞克蓋瑞),\nRe: BOX Lakers 112:103 Clippers 數據\n
9,Mon Mar 9 15:11:57 2020,manuginobii (做愛校 果真累人),\n情報 28次失誤?籃網臨時教練:前7個是球有點滑\n


### ③ 試著爬爬看其他版的文章

### 股票板

In [105]:
#個人備註
#先用成btfs可剖析的變數
#==============================================
from bs4 import BeautifulSoup as btfs
import requests

url ='https://www.ptt.cc/bbs/Stock/index.html'
r = requests.get(url)
soup2 = btfs(r.text,'lxml')
print(soup2.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   看板 Stock 文章列表 - 批踢踢實業坊
  </title>
  <link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>
  <link href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="//images.ptt.cc/bbs/v2.27/bbs-custom.css" rel="stylesheet" type="text/css"/>
  <link href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <div id="topbar-container">
   <div class="bbs-content" id="topbar">
    <a href="/bbs/" id="logo">
     批踢踢實業坊
    </a>
    <span>
     ›
    </span>
    <a class="board" href="/bbs/Stock/index.html">
     <span class="board-label">
      看板
     </span>
     Stock
    </a>
    <a class="right small" href="

In [106]:
#個人備註
#用pandas顯示
#==============================================
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

empty=[]
for i in soup2.find_all('div',class_='r-ent'):
    
    #下面這一行很關鍵，因為我們如果不進去翻內頁，他的日期就只是X/X，沒有更詳細的時間
    #所以設定一個新的btfs可剖析的變數，他在for迴圈裡得以成功是因為他的內文連結在原始碼中
    #剛好可以一個一個翻出來，再append到空list裡
    #但感覺運行起來有點慢，因為要爬取大量的網頁
    
    new_r=btfs(requests.get('https://www.ptt.cc/'+i.find('a')['href']).text,'lxml')
    
    #因為發現他會把\t\n這些空行換行指令弄進來，所以要弄個replace('unwanted','wanted')
    title=i.find('div',class_='title').text.replace('[', '').replace(']', '')
    
    
    #下面這個就是從每個內文裡面去抓他的作者資訊，比原本的詳盡，不知道他是否有活動之類的，
    #他有些文章格式跟其他不一樣，避免整個迴圈出現錯誤，用try,except
    #其實這邊用new_r.find就好了，因為作者就是第1項，但考慮到公整，跟下面用一樣的方法
    detailed_author=new_r.find_all('span',class_='article-meta-value')
    try:                          
        author=detailed_author[0].text.replace('[', '').replace(']', '')
    except:
        author='NaN'
    
    #下面這個就是從每個內文裡面去抓他的日期，比原本的詳盡，不知道他是否有活動之類的，
    #他有些文章格式跟其他不一樣，避免整個迴圈出現錯誤，用try,except
    detailed_date=new_r.find_all('span',class_='article-meta-value')
    try:                          
        date=detailed_date[3].text.replace('[', '').replace(']', '')
    except:
        date='NaN'
    
    #最後就是新增到空list
    empty.append([date,author,title])

    
columns=['日期','作者','標題']#改名
data=pd.DataFrame(empty,columns=columns)#改名
data

,日期,作者,標題
0,Mon Mar 9 19:36:53 2020,iceberg (冰山先生),\nRe: 心得 畢業文 從此不買石油股\n
1,Mon Mar 9 19:37:36 2020,PChome24h (冒牌PChome24h購物),\n新聞 信心喊話 金管會：台股相對抗跌 高殖利率\n
2,Mon Mar 9 19:39:05 2020,chinaeatshit (我愛台灣！中國吃屎！！),\n其他 (2337)旺宏-董事會通過2020年股利分派案\n
3,Mon Mar 9 19:49:51 2020,aaaazzzz (找英文家教),\nRe: 請益 富邦VIX 00677U 後續操作(準畢業)\n
4,Mon Mar 9 20:23:39 2020,iami1002 (股阿甘),\n新聞 賭博成癮4危險徵兆你中了嗎？\n
5,Mon Mar 9 20:36:33 2020,bernard2015 (bernard),\n新聞 證交所喊話：台股兩優勢投資人勿慌\n
6,Mon Mar 9 20:37:52 2020,vic6655443 (nivic),\nRe: 情報 上海發生兩起聚集性疫情\n
7,Mon Mar 9 20:38:22 2020,apple30 (guiguigk),\n請益 如何查美股ETF的溢價幅度？\n
8,Mon Mar 9 20:39:39 2020,vipha (日安早安),\nRe: 心得 現在還有人會覺得股市0風險嗎？\n
9,Mon Mar 9 20:49:52 2020,steward135 (逆風高飛),\n其他 109年03月09日信用交易統計\n
